In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../data/preprocessed_data_points.csv')
print(df.shape)
df.head(2)

In [ ]:
en = "energy-kj_100g"
su = "sugars_100g"
fa = "saturated-fat_100g"
sa = "sodium_100g"
pr = "proteins_100g"
fi = "fiber_100g"
fr = "fruits-vegetables-nuts-estimate-from-ingredients_100g"

profiles = df.loc[:,[en, su, fa, sa, pr, fi, fr]]
print(profiles.shape)
profiles.head(2)

In [ ]:
# Getting quantiles using numpy.percentile() for distinct columns values
quantiles = {}
for col in profiles.columns:
    quantiles[col] = np.percentile(profiles[col].unique(), [0, 20, 40, 60, 80, 100])

# Plotting distribution curves for each column
for col in profiles.columns:
    plt.figure(figsize=(8, 5))
    sns.histplot(profiles[col], kde=True, bins=30, color='green', stat='density')
    plt.title(f'Distribution of {col}')
    plt.xlabel('Values')
    plt.ylabel('Density')  
    plt.show()

In [ ]:
profile_dict = {}

# Creating a dictionary of criteria for each column
for col in profiles.columns:
    each_criteria = []
    for i in range(5):
        each_criteria.append((quantiles[col][i], quantiles[col][i+1]))
    profile_dict[col] = each_criteria

print(profile_dict)

In [ ]:
weights = [
    {en: 1, su: 1, fa: 1, sa: 1, pr: 2, fi: 2, fr: 2},
    # to add one more set of own weights{en: 1, su: 1, fa: 1, sa: 1, pr: 5, fi: 5, fr: 5}
]

In [ ]:
def pessimistic_model():
    return